In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import json

C:\Users\taoiz\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Definir el modelo de diagnóstico
model = BayesianNetwork([('Fever', 'Diagnosis'), ('Cough', 'Diagnosis'), ('Fatigue', 'Diagnosis'), ('ShortnessOfBreath', 'Diagnosis')])

In [3]:
# Definir las probabilidades condicionales
cpd_fever = TabularCPD(variable='Fever', variable_card=2, values=[[0.6], [0.4]])  # Sí/No
cpd_cough = TabularCPD(variable='Cough', variable_card=2, values=[[0.5], [0.5]])
cpd_fatigue = TabularCPD(variable='Fatigue', variable_card=2, values=[[0.7], [0.3]])
cpd_shortness_of_breath = TabularCPD(variable='ShortnessOfBreath', variable_card=2, values=[[0.8], [0.2]])

In [4]:
# Definir CPD para 'Diagnosis' basado en combinaciones de síntomas (ajustando probabilidades para que cada columna sume 1)
cpd_diagnosis = TabularCPD(variable='Diagnosis', variable_card=5,
                           values=[[0.5, 0.4, 0.4, 0.3, 0.3, 0.2, 0.4, 0.2, 0.4, 0.3, 0.2, 0.3, 0.4, 0.2, 0.3, 0.1],
                                   [0.2, 0.3, 0.2, 0.3, 0.3, 0.4, 0.2, 0.4, 0.2, 0.3, 0.4, 0.3, 0.2, 0.3, 0.2, 0.4],
                                   [0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.1, 0.2, 0.1, 0.2, 0.2, 0.2, 0.1, 0.2, 0.2, 0.2],
                                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.2, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2]],
                           evidence=['Fever', 'Cough', 'Fatigue', 'ShortnessOfBreath'],
                           evidence_card=[2, 2, 2, 2])

model.add_cpds(cpd_fever, cpd_cough, cpd_fatigue, cpd_shortness_of_breath, cpd_diagnosis)

In [5]:
# Verificar el modelo
try:
    model.check_model()
    print("El modelo es válido.")
except ValueError as e:
    print(f"Error al verificar el modelo: {e}")

El modelo es válido.


In [6]:
# Inferencia
infer = VariableElimination(model)

In [7]:
# Función para obtener la entrada del usuario
def obtener_entrada_usuario(opciones, tipo):
    while True:
        try:
            print(f"Selecciona el {tipo}:")
            for i, opcion in enumerate(opciones):
                print(f"{i}: {opcion}")
            opcion = int(input(f"Elige un número entre 0 y {len(opciones) - 1}: "))
            if 0 <= opcion < len(opciones):
                return opcion
            else:
                print(f"{tipo} no válido, ingrese una opción correcta.")
        except ValueError:
            print(f"{tipo} no válido, ingrese un número.")

In [8]:
# Definir opciones para síntomas
fiebre_opciones = ['No', 'Sí']
tos_opciones = ['No', 'Sí']
fatiga_opciones = ['No', 'Sí']
dificultad_respirar_opciones = ['No', 'Sí']

In [9]:
# Obtener entradas del usuario
fiebre_input = obtener_entrada_usuario(fiebre_opciones, 'fiebre')
tos_input = obtener_entrada_usuario(tos_opciones, 'tos')
fatiga_input = obtener_entrada_usuario(fatiga_opciones, 'fatiga')
dificultad_respirar_input = obtener_entrada_usuario(dificultad_respirar_opciones, 'dificultad para respirar')

Selecciona el fiebre:
0: No
1: Sí


Elige un número entre 0 y 1:  1


Selecciona el tos:
0: No
1: Sí


Elige un número entre 0 y 1:  0


Selecciona el fatiga:
0: No
1: Sí


Elige un número entre 0 y 1:  1


Selecciona el dificultad para respirar:
0: No
1: Sí


Elige un número entre 0 y 1:  1


In [10]:
# Realizar consulta en la red bayesiana (la evidencia debe ser los índices)
resultado = infer.query(variables=['Diagnosis'], evidence={
    'Fever': fiebre_input,
    'Cough': tos_input,
    'Fatigue': fatiga_input,
    'ShortnessOfBreath': dificultad_respirar_input
})

posterior_values = resultado.values

In [11]:
# Definir los nombres de las posibles enfermedades
enfermedades = ["Influenza", "COVID-19", "Common Cold", "Bronchitis", "Healthy"]

In [12]:
# Identificar la enfermedad con la mayor probabilidad
indice_max = posterior_values.argmax()
enfermedad_probable = enfermedades[indice_max]


In [13]:
# Asignar nombres de síntomas
fiebre_name = fiebre_opciones[fiebre_input]
tos_name = tos_opciones[tos_input]
fatiga_name = fatiga_opciones[fatiga_input]
dificultad_respirar_name = dificultad_respirar_opciones[dificultad_respirar_input]

In [14]:
# Datos que deseas guardar
posterior_values_dict = {
    "Diagnosis": enfermedad_probable,
    "Fever": fiebre_name,
    "Cough": tos_name,
    "Fatigue": fatiga_name,
    "Shortness of Breath": dificultad_respirar_name
}

In [15]:
# Guardar en un archivo JSON
with open('diagnosis_posterior_values.json', 'w') as json_file:
    json.dump(posterior_values_dict, json_file)

In [16]:
# Imprimir la enfermedad con la mayor probabilidad
print(f"La enfermedad más probable es: {enfermedad_probable}")

La enfermedad más probable es: Influenza
